<a href="https://colab.research.google.com/github/saadz-khan/Text2Image/blob/master/animations_improved.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Setup and Library Installation

from IPython.utils import io
with io.capture_output() as captured:
  !pip3 install torch==1.4.0+cu100 torchvision==0.5.0+cu100 -f https://download.pytorch.org/whl/torch_stable.html
  !pip3 install opencv-python==4.2.0.32
  !pip3 install vispy==0.6.4
  !pip3 install moviepy==1.0.2
  !pip3 install transforms3d==0.3.1
  !pip3 install networkx==2.3
  !sudo apt install sed
  !pip install gradio==2.3.7
  import os

  if os.path.exists('/content/drive/MyDrive/pixray/3d-photo-inpainting'):
    pass
  else:
    !mkdir /content/drive/MyDrive/pixray
    %cd /content/drive/MyDrive/pixray
    !git clone https://github.com/vt-vl-lab/3d-photo-inpainting.git
  %cd /content/drive/MyDrive/pixray/3d-photo-inpainting
  !sh download.sh

  # Switch off off-screen rendering
  
  !sed -i 's/offscreen_rendering: True/offscreen_rendering: False/g' argument.yml

  import os
  os.kill(os.getpid(), 9)

In [ ]:
#@title User Interface 
#@markdown Run the play button and select the link generated down below.

import gradio as gr
from PIL import Image
import torch
import numpy as np
import os
import time
from moviepy.editor import *
import yaml

def generate(image, folder, dest_folder):
  %cd /content/drive/MyDrive/pixray/3d-photo-inpainting
  base_path = '/content/drive/MyDrive/pixray/3d-photo-inpainting'
  torch.cuda.empty_cache()
  !rm -rf {base_path}/image
  !rm -rf {base_path}/video
  !mkdir {base_path}/image
  !mkdir {base_path}/video
  !mkdir {base_path}/results

  i = 0
  ext = ('.png', 'jpg')
  if (folder != '/content/drive/MyDrive/'):
    for f in os.listdir(folder):
      if f.endswith(ext):
        !rm -rf {base_path}/image
        !mkdir {base_path}/image
        image = Image.open(folder+'/'+f)
        image.save(f'{base_path}/image/image{i}.png')
      else:
        continue

      !python main.py --config argument.yml
      while not os.path.exists(f'{base_path}/video/image{i}_dolly-zoom-in.mp4'):
        time.sleep(10)

      !python reverse_video.py /content/drive/MyDrive/pixray/3d-photo-inpainting/video/image{i}_dolly-zoom-in.mp4

      # loading video dsa gfg intro video
      clip1 = VideoFileClip(f"/content/drive/MyDrive/pixray/3d-photo-inpainting/video/image{i}_dolly-zoom-in.mp4")
      clip2 = VideoFileClip(f"/content/drive/MyDrive/pixray/3d-photo-inpainting/video/image{i}_dolly-zoom-in-inverted.mp4") 

      # concatenating both the clips
      final1 = concatenate_videoclips([clip1,clip2,clip1,clip2,clip1,clip2])
      final2 = concatenate_videoclips([clip1,clip2])
    
      #writing the video into a file / saving the combined video
      final1.write_videofile(f"{base_path}/video/image{i}.mp4")
      final2.write_gif(f"{base_path}/video/image{i}.gif")
      i = i + 1
      
    return f'{base_path}/video/image{i-1}.mp4'
      

  else:  
    image = Image.fromarray(image)
    image.save(f'{base_path}/image/image{i}.png')
    
    !python main.py --config argument.yml
    while not os.path.exists(f'{base_path}/video/image{i}_dolly-zoom-in.mp4'):
      time.sleep(10)
    !python reverse_video.py /content/drive/MyDrive/pixray/3d-photo-inpainting/video/image{i}_dolly-zoom-in.mp4

    # loading video dsa gfg intro video
    clip1 = VideoFileClip(f"/content/drive/MyDrive/pixray/3d-photo-inpainting/video/image{i}_dolly-zoom-in.mp4")
    clip2 = VideoFileClip(f"/content/drive/MyDrive/pixray/3d-photo-inpainting/video/image{i}_dolly-zoom-in-inverted.mp4") 

    # concatenating both the clips
    final = concatenate_videoclips([clip1,clip2,clip1,clip2,clip1,clip2])
    
    #writing the video into a file / saving the combined video
    final.write_videofile(f"{base_path}/video/image{i}.mp4")
    final.write_gif(f"{base_path}/video/image{i}.gif")
    i = i + 1
    !mv  -v /{base_path}/video/* /{base_path}/{dest_folder}
    return f'/{base_path}/{dest_folder}/image{i-1}.mp4'

# Create the UI
image = gr.inputs.Image(label = 'Image Upload', optional=True)
folder = gr.inputs.Textbox(label = 'Image folder to animate', default = '/content/drive/MyDrive/')
dest_folder = gr.inputs.Textbox(label = 'Destination Folder Name')
#size = gr.inputs.Number(label="Longest side size", default=2000)


# Launch the demo
iface = gr.Interface(generate, inputs=[image, folder, dest_folder], outputs=['video'], enable_queue=True, live=False, title='Animations')
iface.launch(debug=True)

/usr/local/lib/python3.7/dist-packages/paramiko/transport.py:236: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


IMPORTANT: You are using gradio version 2.3.7, however version 2.5.1 is available, please upgrade.
--------
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().

This share link expires in 72 hours. For free permanent hosting, check out Spaces (https://huggingface.co/spaces)
Running on public URL: https://23370.gradio.app
Interface loading below...


/content/drive/MyDrive/pixray/3d-photo-inpainting


[2022-05-04 16:37:42,582] ERROR in app: Exception on /api/queue/status/ [POST]
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/lib/python3.7/dist-packages/flask_cors/extension.py", line 165, in wrapped_function
    return cors_after_request(app.make_response(f(*args, **kwargs)))
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/usr/local/lib/python3.7/dist-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1936, in disp

running on device 0
  0% 0/1 [00:00<?, ?it/s]Current Source ==>  image0
Running depth extraction at 1651682270.0423818
BoostingMonocularDepth/inputs/*.png
BoostingMonocularDepth/outputs/*.png
/usr/local/lib/python3.7/dist-packages/paramiko/transport.py:236: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,
device: cuda
Namespace(Final=True, R0=False, R20=False, colorize_results=False, data_dir='inputs/', depthNet=0, max_res=inf, net_receptive_field_size=None, output_dir='outputs', output_resolution=1, pix2pixsize=1024, savepatchs=0, savewholeest=0)
----------------- Options ---------------
                    Final: True                          	[default: False]
                       R0: False                         
                      R20: False                         
             aspect_ratio: 1.0                           
               batch_size: 1                             
          checkpoints_dir: ./pix2pix/checkpoints     

Moviepy - Done !
Moviepy - video ready /content/drive/MyDrive/pixray/3d-photo-inpainting/video/image0.mp4
MoviePy - Building file /content/drive/MyDrive/pixray/3d-photo-inpainting/video/image0.gif with imageio.


mv: target '//content/drive/MyDrive/pixray/3d-photo-inpainting/ani-test-1' is not a directory


[2022-05-04 17:05:03,588] ERROR in app: Exception on /api/predict/ [POST]
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/lib/python3.7/dist-packages/flask_cors/extension.py", line 165, in wrapped_function
    return cors_after_request(app.make_response(f(*args, **kwargs)))
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/usr/local/lib/python3.7/dist-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1936, in dispatch_

# Helper

In [ ]:
%%writefile /content/drive/MyDrive/pixray/3d-photo-inpainting/reverse_video.py

from moviepy.editor import VideoFileClip, ImageSequenceClip
import numpy as np
import os
from datetime import timedelta, datetime
from glob import glob
from tqdm import tqdm
import shutil

# i.e if video of duration 30 seconds, saves 10 frame per second = 300 frames saved in total
SAVING_FRAMES_PER_SECOND = 30

def format_timedelta(td):
    """Utility function to format timedelta objects in a cool way (e.g 00:00:20.05) 
    omitting microseconds and retaining milliseconds"""
    result = str(td)
    try:
        result, ms = result.split(".")
    except ValueError:
        return result + ".00".replace(":", "-")
    ms = int(ms)
    ms = round(ms / 1e4)
    return f"{result}.{ms:02}".replace(":", "-")


def extract_frames(video_file, verbose=1):
    # load the video clip
    video_clip = VideoFileClip(video_file)
    # make a folder by the name of the video file
    filename, _ = os.path.splitext(video_file)
    if not os.path.isdir(filename):
        os.mkdir(filename)
    # if the SAVING_FRAMES_PER_SECOND is above video FPS, then set it to FPS (as maximum)
    saving_frames_per_second = min(video_clip.fps, SAVING_FRAMES_PER_SECOND)
    # if SAVING_FRAMES_PER_SECOND is set to 0, step is 1/fps, else 1/SAVING_FRAMES_PER_SECOND
    step = 1 / video_clip.fps if saving_frames_per_second == 0 else 1 / saving_frames_per_second
    iteration = np.arange(0, video_clip.duration, step)
    if verbose:
        iteration = tqdm(iteration, desc="Extracting video frames")
    # iterate over each possible frame
    for current_duration in iteration:
        # format the file name and save it
        frame_duration_formatted = format_timedelta(timedelta(seconds=current_duration)).replace(":", "-")
        frame_filename = os.path.join(filename, f"frame{frame_duration_formatted}.jpg")
        # save the frame with the current duration
        video_clip.save_frame(frame_filename, current_duration)
    return filename, video_clip.fps



def reverse_video(frames_path, video_fps, remove_extracted_frames=True):
    frame_files = glob(os.path.join(frames_path, "*"))
    # sort by duration in descending order
    frame_files = sorted(frame_files, key=lambda d: datetime.strptime(d.split("frame")[1], "%H-%M-%S.%f.jpg"), reverse=True)
    # calculate the FPS, getting the minimum between the original FPS and the parameter we set
    saving_frames_per_second = min(video_fps, SAVING_FRAMES_PER_SECOND)
    if saving_frames_per_second == 0:
        # if the parameter is set to 0, automatically set it to the original video fps
        saving_frames_per_second = video_fps
    print("Saving the video with FPS:", saving_frames_per_second)
    # load the frames into a image sequence clip (MoviePy)
    image_sequence_clip = ImageSequenceClip(frame_files, fps=saving_frames_per_second)
    # write the video file to disk
    output_filename = f"{frames_path}-inverted.mp4"
    image_sequence_clip.write_videofile(output_filename)
    if remove_extracted_frames:
        # if set to True, then remove the folder that contain the extracted frames
        shutil.rmtree(frames_path)



if __name__ == "__main__":
    import sys
    video_file = sys.argv[1]
    frames_folder_path, video_fps = extract_frames(video_file)
    reverse_video(frames_folder_path, video_fps=video_fps)

Writing /content/drive/MyDrive/pixray/3d-photo-inpainting/reverse_video.py


In [ ]:
filename = '/content/drive/MyDrive/pixray/3d-photo-inpainting/video/final_dolly-zoom-in.mp4' #@param{type:'string'}
!python /content/reverse_video.py {filename} 

Extracting video frames: 100% 181/181 [00:03<00:00, 48.15it/s]
Saving the video with FPS: 30
Moviepy - Building video /content/drive/MyDrive/pixray/3d-photo-inpainting/video/final_dolly-zoom-in-inverted.mp4.
Moviepy - Writing video /content/drive/MyDrive/pixray/3d-photo-inpainting/video/final_dolly-zoom-in-inverted.mp4

Moviepy - Done !
Moviepy - video ready /content/drive/MyDrive/pixray/3d-photo-inpainting/video/final_dolly-zoom-in-inverted.mp4


In [ ]:
# Import everything needed to edit video clips
from moviepy.editor import *
 
# loading video dsa gfg intro video
clip1 = VideoFileClip("/content/drive/MyDrive/pixray/3d-photo-inpainting/video/final_dolly-zoom-in.mp4")
clip2 = VideoFileClip("/content/drive/MyDrive/pixray/3d-photo-inpainting/video/final_dolly-zoom-in-inverted.mp4") 

# concatenating both the clips
final = concatenate_videoclips([clip1,clip2,clip1,clip2,clip1,clip2])
#writing the video into a file / saving the combined video
final.write_videofile("/content/merged.mp4")

Moviepy - Building video /content/merged.mp4.
Moviepy - Writing video /content/merged.mp4



Moviepy - Done !
Moviepy - video ready /content/merged.mp4


In [ ]:
!ffmpeg -stream_loop 5 -i /content/merged.mp4 -c copy /content/output.mp4

ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lib

In [ ]:
!python main.py --config argument.yml


running on device 0
  0% 0/2 [00:00<?, ?it/s]Current Source ==>  image0
Running depth extraction at 1648581130.2973347
BoostingMonocularDepth/inputs/*.png
BoostingMonocularDepth/outputs/*.png
device: cuda
Namespace(Final=True, R0=False, R20=False, colorize_results=False, data_dir='inputs/', depthNet=0, max_res=inf, net_receptive_field_size=None, output_dir='outputs', output_resolution=1, pix2pixsize=1024, savepatchs=0, savewholeest=0)
----------------- Options ---------------
                    Final: True                          	[default: False]
                       R0: False                         
                      R20: False                         
             aspect_ratio: 1.0                           
               batch_size: 1                             
          checkpoints_dir: ./pix2pix/checkpoints         
         colorize_results: False                         
                crop_size: 672                           
                 data_dir: inputs/    

In [ ]:
import yaml

with open('/content/drive/MyDrive/pixray/3d-photo-inpainting/argument.yml') as file:
    documents = yaml.load(file)

    for item, doc in documents.items():
        print(item, ":", doc)

depth_edge_model_ckpt : checkpoints/edge-model.pth
depth_feat_model_ckpt : checkpoints/depth-model.pth
rgb_feat_model_ckpt : checkpoints/color-model.pth
MiDaS_model_ckpt : MiDaS/model.pt
use_boostmonodepth : True
fps : 40
num_frames : 240
x_shift_range : [0.0, 0.0, -0.015, -0.015]
y_shift_range : [0.0, 0.0, -0.015, -0.0]
z_shift_range : [-0.05, -0.05, -0.05, -0.05]
traj_types : ['double-straight-line', 'double-straight-line', 'circle', 'circle']
video_postfix : ['dolly-zoom-in', 'zoom-in', 'circle', 'swing']
specific : 
longer_side_len : 2000
src_folder : image
depth_folder : depth
mesh_folder : mesh
video_folder : video
load_ply : False
save_ply : True
inference_video : True
gpu_ids : 0
offscreen_rendering : False
img_format : .png
depth_format : .npy
require_midas : True
depth_threshold : 0.04
ext_edge_threshold : 0.002
sparse_iter : 5
filter_size : [7, 7, 5, 5, 5]
sigma_s : 4.0
sigma_r : 0.5
redundant_number : 12
background_thickness : 70
context_thickness : 140
background_thickness

In [ ]:
import yaml

dict_file = [{'sports' : ['soccer', 'football', 'basketball', 'cricket', 'hockey', 'table tennis']},
{'countries' : ['Pakistan', 'USA', 'India', 'China', 'Germany', 'France', 'Spain']}]

with open(r'E:\data\store_file.yaml', 'w') as file:

    documents = yaml.dump(dict_file, file)8

In [ ]:
yaml_dict = yaml.safe_load('')
print(yaml_dict)

/content/drive/MyDrive/pixray/3d-photo-inpainting/argument.yml


In [ ]:
import yaml
from pathlib import Path
conf = yaml.safe_load(Path('/content/drive/MyDrive/pixray/3d-photo-inpainting/argument.yml').read_text())

In [ ]:
conf

{'MiDaS_model_ckpt': 'MiDaS/model.pt',
 'anti_flickering': True,
 'background_thickness': 70,
 'background_thickness_2': 70,
 'context_thickness': 140,
 'context_thickness_2': 70,
 'crop_border': [0.03, 0.03, 0.05, 0.03],
 'depth_edge_dilate': 10,
 'depth_edge_dilate_2': 5,
 'depth_edge_model_ckpt': 'checkpoints/edge-model.pth',
 'depth_feat_model_ckpt': 'checkpoints/depth-model.pth',
 'depth_folder': 'depth',
 'depth_format': '.npy',
 'depth_threshold': 0.04,
 'discount_factor': 1.0,
 'ext_edge_threshold': 0.002,
 'extrapolate_border': True,
 'extrapolation_thickness': 60,
 'filter_size': [7, 7, 5, 5, 5],
 'fps': 40,
 'gpu_ids': 0,
 'img_format': '.png',
 'inference_video': True,
 'largest_size': 512,
 'load_ply': False,
 'log_depth': True,
 'longer_side_len': 2000,
 'mesh_folder': 'mesh',
 'num_frames': 240,
 'offscreen_rendering': False,
 'redundant_number': 12,
 'repeat_inpaint_edge': True,
 'require_midas': True,
 'rgb_feat_model_ckpt': 'checkpoints/color-model.pth',
 'save_ply': 

In [ ]:
def change_yaml(largest_size, total_frames, fps, longest_side, path):

  #largest_size = 512
  #total_frames = 240
  #fps = 40
  #longest_side = 2000
  yaml_dict = {'MiDaS_model_ckpt': 'MiDaS/model.pt',
  'anti_flickering': True,
  'background_thickness': 70,
  'background_thickness_2': 70,
  'context_thickness': 140,
  'context_thickness_2': 70,
  'crop_border': [0.03, 0.03, 0.05, 0.03],
  'depth_edge_dilate': 10,
  'depth_edge_dilate_2': 5,
  'depth_edge_model_ckpt': 'checkpoints/edge-model.pth',
  'depth_feat_model_ckpt': 'checkpoints/depth-model.pth',
  'depth_folder': 'depth',
  'depth_format': '.npy',
  'depth_threshold': 0.04,
  'discount_factor': 1.0,
  'ext_edge_threshold': 0.002,
  'extrapolate_border': True,
  'extrapolation_thickness': 60,
  'filter_size': [7, 7, 5, 5, 5],
  'fps': {fps},
  'gpu_ids': 0,
  'img_format': '.png',
  'inference_video': True,
  'largest_size': {int(largest_size)},
  'load_ply': False,
  'log_depth': True,
  'longer_side_len': {int(longest_side)},
  'mesh_folder': 'mesh',
  'num_frames': {int(total_frames)},
  'offscreen_rendering': False,
  'redundant_number': 12,
  'repeat_inpaint_edge': True,
  'require_midas': True,
  'rgb_feat_model_ckpt': 'checkpoints/color-model.pth',
  'save_ply': True,
  'sigma_r': 0.5,
  'sigma_s': 4.0,
  'sparse_iter': 5,
  'specific': '',
  'src_folder': 'image',
  'traj_types': ['double-straight-line',
    'double-straight-line',
    'circle',
    'circle'],
  'use_boostmonodepth': True,
  'video_folder': 'video',
  'video_postfix': ['dolly-zoom-in', 'zoom-in', 'circle', 'swing'],
  'x_shift_range': [0.0, 0.0, -0.015, -0.015],
  'y_shift_range': [0.0, 0.0, -0.015, -0.0],
  'z_shift_range': [-0.05, -0.05, -0.05, -0.05]}

  with open(path, 'w') as file:
    yaml.dump(yaml_dict, file)

In [ ]:
change_yaml(513, 250, 60, 2001, '/content/argument.yml')